In [2]:
import pandas as pd
import numpy as np
import altair as alt

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from time import time

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\manut\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [13]:
df_relatos = pd.read_csv('C:\\Users\\manut\\Downloads\\base_tratada(2019-2023).csv')


In [14]:
total_palavras = df_relatos['relato'].apply(lambda x: len(x.split())).sum()

print(f'Total de palavras no DataFrame: {total_palavras}')

Total de palavras no DataFrame: 6556784


In [15]:
df_relatos.shape

(47607, 21)

In [16]:
relatos=df_relatos[['relato']]

In [17]:
relatos['relato']

0        nacional marisangela ferreira pacheco uipp - t...
1        relatora qualificada , seccional urbana políci...
2        , especializada denunciar irmão ailton silva f...
3        relator qualificado , qualidade policial milit...
4        relatora qualificada epígrafe seccional comuni...
                               ...                        
47602    relatora qualificada comparece seccional polic...
47603    casada natanael dantas rocha junior , nascido ...
47604    relatora seccional policia civil sao bras comu...
47605    relatora qualificada comparece seccional polic...
47606    relatora qualificada comparece especializada c...
Name: relato, Length: 47607, dtype: object

In [18]:
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['fica', 'fim', 'ato', 'desde', 'pois', 'abadessa',
                   'abaetetuba', 'abaetetuba abaetetuba', 'abaetetuba contatos',
                   'abaixa', 'abaixada', 'abaixado', 'abaixar','abaixar som', 'abaixar volume',
                   'abaixasse', 'abaixasse som', 'abaixo', 'abaixo assinado','abaixo elencadas', 
                   'abaixo informacoes', 'abaixo olho','abaixo proibicao', 'abaixo proibição', 
                   'abaixou', 'abaixou pegar','abaixou vidro',])

stop_words = list(stop_words)


relatos_limpos = []
for w in range(len(relatos)):
  Relato = relatos['relato'].iloc[w]

  Relato  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",Relato )
  Relato = ' '.join(s for s in Relato.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
  relatos_limpos.append(Relato)


relatos_limpos[0:5]

['nacional marisangela ferreira pacheco uipp terra firme comunicar citados trabalhando esposa funcinário trabalhar chamada nezia chaves gomes fazendo escândalo frente loja relatora trabalha mulher dizia vou pegar cortar toda cara desa puta vou deixar arreada chão furar todinha textuais vítima diz tempo nezia espalha relatora caso marido levy baia relatora trabalha sente constrangida fofocas ameaças tempo pede levy convencer esposa dois relação rapaz faz nada fica calado sabe suspeita reside marambaia passagem simão jatene número água cristal trabalha passagem francisco xavier final jabatiteua padaria fica esquina próximo vileta',
 'relatora qualificada seccional urbana polícia declarar enquanto visitava tia nacional geraldina brito sales hospital saúde mulher localizada humaita marco agredida fisicamente tapas arranhões puxões socos região rosto tórax membros superiores nacional simone carla sales duarte prima contenda começou devido problemas familiares relatora revidou procurou afast

* Processo de vetorização

In [19]:
tf_vectorizer = CountVectorizer(
    min_df=5,           # Considera palavras que aparecem em pelo menos 5 documentos
    max_df=0.5,          # Exclui termos que aparecem em mais de 50% dos documentos
    max_features=100000,  # Limita o número de termos no vocabulário aos 100.000 mais frequentes
    stop_words=stop_words, # Remove palavras comuns que não são úteis
    ngram_range=(1, 2)   # Considera uni-gramas (termos únicos) e bi-gramas (pares de termos)
)

#Transformação
vec_text = tf_vectorizer.fit_transform(relatos_limpos)

#Retorna a lista de palavras
words = tf_vectorizer.get_feature_names_out()

print(vec_text.shape)
print(len(words))

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['assinado', 'contatos', 'elencadas', 'informacoes', 'olho', 'pegar', 'proibicao', 'proibição', 'som', 'vidro', 'volume'] not in stop_words.
  warnings.warn(


(47607, 100000)
100000


In [20]:
words[0:20]

array(['abala', 'abalada', 'abalada chorando', 'abalada declarante',
       'abalada devido', 'abalada emocionalmente', 'abalada medo',
       'abalada psicologicamente', 'abalada situacao', 'abalada teme',
       'abaladas', 'abalado', 'abalado psicologico', 'abalados',
       'abalando', 'abalando psicologico', 'abalar', 'abalda', 'abalo',
       'abalo emocional'], dtype=object)

* Encontrar os tópicos

In [21]:
from sklearn.decomposition import LatentDirichletAllocation

In [22]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("\n--\nTopic #{}: ".format(topic_idx + 1))
    message = ", ".join([feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()

def display_topics(W, H, feature_names, relatos, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx:
            print('Relato índice {} : \t{:.2f}'.format(d, W[d, topic_idx]))

In [23]:
lda = LatentDirichletAllocation(n_components=20, 
                                learning_method='batch', # 'online' equivale a minibatch no k-means
                                random_state=0)

t0 = time()

lda.fit(vec_text)
doc_topic_matrix = lda.transform(vec_text)

print("done in %0.3fs." % (time() - t0))

done in 674.724s.


* Palavras mais associadas com cada tópico

In [24]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names_out()
print_top_words(lda, tf_feature_names, 20)


Topics in LDA model:

--
Topic #1: 
vitima, local, policial, militar, jurunas, silva, vtr, ciop, santos, bairro, fato, acusado, via, havia, guarnicao, relator, sido, policia, sgt, rua

--
Topic #2: 
nacional, seccional, icoaraci, urbana, seccional urbana, local, fato, companheiro, silva, carlos, comunicar, bairro, qualificada, sacramenta, rua, qualificada seccional, marambaia, passagem, momento, relatante

--
Topic #3: 
medidas, protetivas, medidas protetivas, companheiro, deseja, relacionamento, filhos, filho, criminalmente, comunicante, contra, cpf, especializada, abrigo, silva, vai, representar, bairro, meses, nascido

--
Topic #4: 
vou, textuais, filha, nacional, vai, residencia, ameaca, rua, mae, ameacas, comunicar, bairro, seguintes, maria, vem, irmao, palavras, vitima, sra, silva

--
Topic #5: 
ofendida, proibicao, ofendida familiares, familiares, familiares testemunhas, testemunhas, contato, agressor, contato ofendida, limite, qualquer, frequentar, preservar, preservar integri

* Documentos mais associados a cada topico

In [25]:
display_topics(doc_topic_matrix, lda.components_, tf_feature_names, relatos, 15, 10)



--
Topic #1: 
VITIMA, LOCAL, POLICIAL, MILITAR, JURUNAS, SILVA, VTR, CIOP, SANTOS, BAIRRO, FATO, ACUSADO, VIA, HAVIA, GUARNICAO
Relato índice 46236 : 	0.99
Relato índice 20186 : 	0.99
Relato índice 45953 : 	0.99
Relato índice 45954 : 	0.99
Relato índice 37598 : 	0.99
Relato índice 37033 : 	0.99
Relato índice 35298 : 	0.99
Relato índice 46848 : 	0.99
Relato índice 41540 : 	0.99
Relato índice 40374 : 	0.99

--
Topic #2: 
NACIONAL, SECCIONAL, ICOARACI, URBANA, SECCIONAL URBANA, LOCAL, FATO, COMPANHEIRO, SILVA, CARLOS, COMUNICAR, BAIRRO, QUALIFICADA, SACRAMENTA, RUA
Relato índice 24085 : 	0.99
Relato índice 12875 : 	0.94
Relato índice 19521 : 	0.91
Relato índice 37192 : 	0.91
Relato índice 20995 : 	0.90
Relato índice 29567 : 	0.90
Relato índice 40004 : 	0.88
Relato índice 33894 : 	0.87
Relato índice 33935 : 	0.87
Relato índice 13788 : 	0.87

--
Topic #3: 
MEDIDAS, PROTETIVAS, MEDIDAS PROTETIVAS, COMPANHEIRO, DESEJA, RELACIONAMENTO, FILHOS, FILHO, CRIMINALMENTE, COMUNICANTE, CONTRA, CPF, E

In [26]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(47607, 20)
Matriz tópicos-termos:(20, 100000)


In [27]:
main_topic = []
mt_prob = []
for l in range(len(relatos['relato'])):
  main_topic.append(doc_topic_matrix[l, :].argmax() + 1)
  mt_prob.append(doc_topic_matrix[l, :].max())

relatos = relatos.assign(main_topic=main_topic, main_topic_prob=mt_prob)


In [28]:
topico = 2
pd.options.display.max_colwidth = 300
relatos[relatos['main_topic'] == topico].sort_values('main_topic_prob', ascending = False)[['main_topic_prob','relato']].head(20).sample(10)

,main_topic_prob,relato
20995,0.903261,"seccional icoaraci relatar agredida fisica mente caminhava via publica nacional pre elizabeth conhecida beth esperando local proximidades conjunto citado , beth faca , declarante conseguiu puxar faca maoes , agredida beth rosto , caindo chao puxou cabelo declarante , motivado beth declarante and..."
12875,0.942996,"seccional urbana polícia ( ) nacional silvana lucia barbosa silva , registrar , constantes neste bopc , ameaçada nacional rogerio , vizinho atirou tijolo filho 08 , exaltou ânimos partes . registra-se"
13788,0.873577,"/// relatora qualificada seccional urbana icoaraci , comunicar supracitados surpreendida nacionais telma amador , diene amador , delma amador alexandra , parentes esposo declarante , passaram ameaçar relatora morte , seguida partiram vias fato contra relatora , telma amador ofendeu partiu vias f..."
6168,0.811794,"declarante qualificada trabalha artesã 08:30hs encontrava-se encontrava-se companhia esposo sr. haroldo sebastião silva , ambos residentes rua vitor hugo , casa , _____ , invasão grão pará , bairro ariramba , ilha mosqueiro , lá chegou filho fabricio josé maciel silva , nº 6423529 brasileiro , p..."
24008,0.840127,"/////// seccional relatora manoeli sousa oliveira , comunicar & vitima agressoes fisicas desferidas ex companheiro gerson rodrigues alencar contato : ( 91 ) 98464-7082 , aduz relatora ambos encontram separados desde outubro 2020 permanece frequentando residencia amigavelmente & 27/06/2021 ( domi..."
33894,0.874577,"/////////////////// declarante paulo renato barata silva , seccional urbana icoaraci presente comunicar & 05/10/2022 03:30hs , companhia beatriz mendes santos , angela franco pantoja , jair silva nascimento nascimento larissa barbosa silva , momento repouso terreno ocupado mesmos localizado rua ..."
24432,0.832867,"declarante identificada seccional urbana marambaia comunicar vitima agressoes fisicas verbais filho bruno ferreira silva , nora darli almeida irmao rovane ferreira silva . ouve principio discussao devido supostos barulhos feitos logo cedo declarante acordado filho irritado . discussao evolui tro..."
19521,0.911124,"¿////////////////////////// nacional qualificada seccional supracitados comunicar vitima agressoes fisicas desferidas ex companheiro luiz pereira cruz 54 ( cinquenta quatro ) rua andradas 8 maio n° 11 ( vila kitnets ) , relata nacional alem agressoes fisicas agrediu palavras & fato & via publica..."
28473,0.847871,declarante identificada seccional urbana marambaia comunicar agredida socos ameacada nacional prenome “ angela ” atual companheira pai filhos . relatante casa ex-companheiro deixar filhos menores cuidado pai enquanto iria realizar algumas compras fato desagradou “ angela ” iniciou desentendiment...
35020,0.866909,"nacional devidamente qualificada 1ªseccional urbana sacramenta registrar citados vitima agressao , discutindo ex-companheiro , hugo portal ribeiro , reside passagem marinho nº05 rua nova travessa barao triunfo , veio dar tapa rosto , fato 21/05/2022 , 02/06/2022 , nova discursarao aonde vieram c..."


Topicos por documento

In [49]:
# Nomes das colunas dos tópicos
topicnames = ["Topic {}".format(i + 1) for i in range(doc_topic_matrix.shape[1])]

# Criar DataFrame relato_topico
relato_topico = pd.DataFrame(np.round(doc_topic_matrix, 2), columns=topicnames, index=relatos.index)

# Adicionar o índice do relato ao DataFrame relato_topico
relato_topico['Relato_Index'] = relatos.index

# Reorganizar as colunas
ordem = ['Relato_Index']
ordem.extend(topicnames)
relato_topico = relato_topico[ordem]

# Visualizar o DataFrame relato_topico
relato_topico.head()



,Relato_Index,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,...,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19,Topic 20
0,0,0.00,0.00,0.00,0.32,0.00,0.0,0.00,0.00,0.05,...,0.0,0.0,0.32,0.16,0.00,0.0,0.0,0.00,0.0,0.0
1,1,0.00,0.16,0.00,0.00,0.00,0.0,0.09,0.42,0.00,...,0.0,0.0,0.06,0.26,0.00,0.0,0.0,0.00,0.0,0.0
2,2,0.00,0.00,0.00,0.04,0.32,0.0,0.00,0.22,0.02,...,0.0,0.0,0.00,0.00,0.06,0.0,0.0,0.01,0.0,0.0
3,3,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,...,0.0,0.0,0.00,0.20,0.80,0.0,0.0,0.00,0.0,0.0
4,4,0.02,0.00,0.15,0.00,0.00,0.0,0.08,0.00,0.00,...,0.0,0.0,0.00,0.52,0.00,0.0,0.0,0.05,0.0,0.0


* Visualizações

In [41]:
import pyLDAvis
from pyLDAvis import lda_model
pyLDAvis.enable_notebook()

In [42]:
pyLDAvis.lda_model.prepare(lda, vec_text, tf_vectorizer, sort_topics=False, mds = 'tsne')

PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      36.812817  22.029352       1        1   3.147057
1      30.707607 -16.143711       2        1   2.607376
2     -44.550827  57.628929       3        1   6.159145
3      70.344772 -10.540270       4        1   8.866753
4     -88.974564 -13.150914       5        1   8.806057
5       4.990261   9.588365       6        1   5.085199
6     -30.487507 -47.691322       7        1   5.675106
7      -7.698317  81.488411       8        1   6.079877
8     -50.483833 -13.353112       9        1   1.738279
9     -31.839996  18.295118      10        1  13.023306
10    -13.965113 -16.630945      11        1   2.931788
11     53.832790 -50.200138      12        1   3.132842
12     72.739899  34.304508      13        1   3.803087
13    -71.956558  26.949017      14        1   3.743095
14    -22.213610 -84.542068      15        1   2.739128
15     22.442959 -80.391243      16        1   1.493735
16     -2.877559  43.182701      17        1   9.474207
17      8.471931 -44.573051      18        1   3.429189
18     34.539005  62.989635      19        1   4.579793
19    -68.617653 -55.510120      20        1   3.484979, topic_info=             Term          Freq         Total Category  logprob  loglift
26446  declarante  51812.000000  51812.000000  Default  30.0000  30.0000
28863    depoente  13677.000000  13677.000000  Default  29.0000  29.0000
64105    ofendida  25300.000000  25300.000000  Default  28.0000  28.0000
60216    nacional  26692.000000  26692.000000  Default  27.0000  27.0000
72151   proibicao  13671.000000  13671.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
52872       local    606.465624  19990.189476  Topic20  -5.9907  -0.1386
60216    nacional    612.348078  26692.479207  Topic20  -5.9810  -0.4181
43210      fisica    512.956762   7609.457214  Topic20  -6.1581   0.6598
46410       havia    506.997771   7790.045837  Topic20  -6.1698   0.6246
58261     momento    520.195594  11478.350113  Topic20  -6.1441   0.2627

[1688 rows x 6 columns], token_table=       Topic      Freq                    Term
term                                          
139        4  0.017781                  abraco
139        7  0.960167                  abraco
139       20  0.017781                  abraco
175       17  0.998275  abrigamento temporario
178        1  0.000680                  abrigo
...      ...       ...                     ...
99717     12  0.990124            www facebook
99750      5  0.989982     xingada companheiro
99892      2  0.945638              ªseccional
99892     15  0.029551              ªseccional
99894      2  0.963878       ªseccional urbana

[7998 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])

In [43]:
from sklearn.manifold import TSNE

relatos_embedded = TSNE(n_components=2, verbose=1, perplexity=40, early_exaggeration=20).fit_transform(doc_topic_matrix)

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 47607 samples in 0.002s...
[t-SNE] Computed neighbors for 47607 samples in 5.888s...
[t-SNE] Computed conditional probabilities for sample 1000 / 47607
[t-SNE] Computed conditional probabilities for sample 2000 / 47607
[t-SNE] Computed conditional probabilities for sample 3000 / 47607
[t-SNE] Computed conditional probabilities for sample 4000 / 47607
[t-SNE] Computed conditional probabilities for sample 5000 / 47607
[t-SNE] Computed conditional probabilities for sample 6000 / 47607
[t-SNE] Computed conditional probabilities for sample 7000 / 47607
[t-SNE] Computed conditional probabilities for sample 8000 / 47607
[t-SNE] Computed conditional probabilities for sample 9000 / 47607
[t-SNE] Computed conditional probabilities for sample 10000 / 47607
[t-SNE] Computed conditional probabilities for sample 11000 / 47607
[t-SNE] Computed conditional probabilities for sample 12000 / 47607
[t-SNE] Computed conditional probabilities for sa

In [51]:
relatos = relatos.assign(tsne1 = relatos_embedded[:,0], tsne2 = relatos_embedded[:,1])

alt.Chart(relatos.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'main_topic:N',
    size = 'main_topic_prob',
    tooltip = ['relato', 'main_topic', 'main_topic_prob']
).interactive()

alt.Chart(...)

* Agrupamento de acordo com os topicos

In [52]:
from sklearn.cluster import MiniBatchKMeans

kmeans = MiniBatchKMeans(n_clusters=11, init_size=1024, batch_size=2048, random_state=20)
#fit the data 
kmeans.fit(doc_topic_matrix)
labels = kmeans.predict(doc_topic_matrix)

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 3 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [53]:
relatos1 = relato_topico.assign(grupo = labels, 
                              tsne1 = relatos_embedded[:,0], tsne2 = relatos_embedded[:,1])

alt.Chart(relatos1.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'grupo:N',
    tooltip = [str(c) for c in relatos1.columns]
).interactive()

alt.Chart(...)

In [56]:
indice_desejado = 42

alt.Chart(relatos.query('Indice == @indice_desejado')).mark_circle(
    opacity=0.7,
    size=30
).encode(
    x='tsne1',
    y='tsne2',
    color='main_topic:N',
    size='main_topic_prob',
    tooltip=['relato', 'main_topic', 'main_topic_prob']
).interactive()

UndefinedVariableError: name 'Indice' is not defined